##### ML jupyter notebook
##### This notebook is dedicated to my finding trends and analysis of accidents

In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import os

In [2]:
# see if we have the right file in working directory
for files in os.listdir():
    print(files)

.git
.ipynb_checkpoints
Capstone.ipynb
Data-Collisions.csv
Location.ipynb
README.md
TestFile.txt
US_Accidents_June20.csv
WebApp_JupyterNB.ipynb


# Need to drop some columns cause of recurring memory issues
#### "Sunrise_Sunset" shows the period of day (i.e day or night) based on sunrise/sunset.
#### "Visibility(mi)" is being measured in miles

In [3]:
# useful columns
useful_colmns = ["Source", "Severity", "Start_Time", "End_Time", "City", "County", "State", 
              "Visibility(mi)", "Weather_Condition", "Sunrise_Sunset"]

df = pd.read_csv("US_Accidents_June20.csv", usecols=useful_colmns)

In [4]:
df.rename(columns={'Visibility(mi)': 'Visibility'}, inplace = True)
df.head(10)

,Source,Severity,Start_Time,End_Time,City,County,State,Visibility,Weather_Condition,Sunrise_Sunset
0,MapQuest,3,2016-02-08 05:46:00,2016-02-08 11:00:00,Dayton,Montgomery,OH,10.0,Light Rain,Night
1,MapQuest,2,2016-02-08 06:07:59,2016-02-08 06:37:59,Reynoldsburg,Franklin,OH,10.0,Light Rain,Night
2,MapQuest,2,2016-02-08 06:49:27,2016-02-08 07:19:27,Williamsburg,Clermont,OH,10.0,Overcast,Night
3,MapQuest,3,2016-02-08 07:23:34,2016-02-08 07:53:34,Dayton,Montgomery,OH,9.0,Mostly Cloudy,Night
4,MapQuest,2,2016-02-08 07:39:07,2016-02-08 08:09:07,Dayton,Montgomery,OH,6.0,Mostly Cloudy,Day
5,MapQuest,3,2016-02-08 07:44:26,2016-02-08 08:14:26,Westerville,Franklin,OH,7.0,Light Rain,Day
6,MapQuest,2,2016-02-08 07:59:35,2016-02-08 08:29:35,Dayton,Montgomery,OH,7.0,Overcast,Day
7,MapQuest,3,2016-02-08 07:59:58,2016-02-08 08:29:58,Dayton,Montgomery,OH,7.0,Overcast,Day
8,MapQuest,2,2016-02-08 08:00:40,2016-02-08 08:30:40,Dayton,Montgomery,OH,5.0,Mostly Cloudy,Day
9,MapQuest,3,2016-02-08 08:10:04,2016-02-08 08:40:04,Westerville,Franklin,OH,3.0,Light Rain,Day


#### The code below works for reading in chunks but I keep running into memory issues and TextToFileReader problems

In [9]:
# The df contains up to 3.6 million of data 
# chunkSIZE = 500000 # we're reading our data in chunks because of how large our data set is
# dataList = [] ## list to hold the chunks for concatenation 

# # for chunk in pd.read_csv("US_Accidents_June20.csv", chunksize = chunkSIZE):
# #     dataList.append(chunk)

# # df = pd.concat(dataList, axis=0)
# chunk1 = pd.read_csv("US_Accidents_June20.csv", chunksize = 1000)

#### We need to do some filtering and preprocessing to our data to get rid of redundant entries

In [5]:
df.shape

(3513617, 10)

In [6]:
# Steps in dealing with missing data values
missing_data = df.isnull()
missing_data.head(10) ## checking the tail to see if any data is missing

,Source,Severity,Start_Time,End_Time,City,County,State,Visibility,Weather_Condition,Sunrise_Sunset
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False


In [8]:
# for each column in the df, we wanna look through to find any missing data
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")

Source
False    3513617
Name: Source, dtype: int64

Severity
False    3513617
Name: Severity, dtype: int64

Start_Time
False    3513617
Name: Start_Time, dtype: int64

End_Time
False    3513617
Name: End_Time, dtype: int64

City
False    3513505
True         112
Name: City, dtype: int64

County
False    3513617
Name: County, dtype: int64

State
False    3513617
Name: State, dtype: int64

Visibility
False    3437761
True       75856
Name: Visibility, dtype: int64

Weather_Condition
False    3437479
True       76138
Name: Weather_Condition, dtype: int64

Sunrise_Sunset
False    3513502
True         115
Name: Sunrise_Sunset, dtype: int64



Based on the output above, ONLY 4 columns containmissing data including
City, Visibility(mi), Weather_Condition, Sunrise_Sunset

## Deal with missing data

##### How to deal with missing data?
 ###### 1. drop data
   a) drop the whole row
   b) drop the whole column
 ###### 2. replace data
   a) replace it by mean
   b) replace it by frequency
   c) replace it based on other functions
   
##### Replace by mean
##### Replace by frequency
##### Drop the whole row

In [9]:
# Replace by frequency for City column
df['City'].value_counts()

Houston                101240
Los Angeles             79169
Charlotte               78952
Dallas                  64790
Austin                  63889
                        ...  
Cat Spring                  1
Union Park                  1
Bridgewater Corners         1
Wentworths Location         1
Westmorland                 1
Name: City, Length: 11895, dtype: int64

In [10]:
df['City'].value_counts().idxmax()

'Houston'

#### Since "Houston" has the most occurence for the City column, we replace 112 missing data with "Houston"
making the new total for "Houston" 101352

In [11]:
# replace the missing 'City' values by the most frequent 
df["City"].replace(np.nan, "Houston", inplace=True)

In [12]:
df['City'].value_counts()

Houston                101352
Los Angeles             79169
Charlotte               78952
Dallas                  64790
Austin                  63889
                        ...  
Cat Spring                  1
Union Park                  1
Bridgewater Corners         1
Wentworths Location         1
Westmorland                 1
Name: City, Length: 11895, dtype: int64

<h4>Calculate the mean value for the  'Visibility' column:</h4>

In [13]:
avg_Visibility = df['Visibility'].astype('float').mean(axis=0)
print("Average Visibility:", avg_Visibility)

Average Visibility: 9.122644290862572


<h4>Replace "NaN" by mean value:</h4>

In [14]:
df['Visibility'].replace(np.nan, avg_Visibility, inplace=True)

##### Checking if preprocessing has updated in our original dataframe

In [15]:
missing_data2 = df.isnull()
for column in missing_data2.columns.values.tolist():
    print(column)
    print(missing_data2[column].value_counts())
    print("")

Source
False    3513617
Name: Source, dtype: int64

Severity
False    3513617
Name: Severity, dtype: int64

Start_Time
False    3513617
Name: Start_Time, dtype: int64

End_Time
False    3513617
Name: End_Time, dtype: int64

City
False    3513617
Name: City, dtype: int64

County
False    3513617
Name: County, dtype: int64

State
False    3513617
Name: State, dtype: int64

Visibility
False    3513617
Name: Visibility, dtype: int64

Weather_Condition
False    3437479
True       76138
Name: Weather_Condition, dtype: int64

Sunrise_Sunset
False    3513502
True         115
Name: Sunrise_Sunset, dtype: int64



In [16]:
# Replace by frequency for Weather_Condition column
df['Weather_Condition'].value_counts()

Clear                        808202
Fair                         547721
Mostly Cloudy                488094
Overcast                     382485
Partly Cloudy                344815
                              ...  
Drifting Snow                     1
Freezing Rain / Windy             1
Thunder and Hail                  1
Sand / Dust Whirls Nearby         1
Freezing Drizzle                  1
Name: Weather_Condition, Length: 127, dtype: int64

#### Since "Clear" has the most occurence for the Weather_Condition column, we replace 76138 missing data with "Clear"
making the new total for "Clear" 884340

In [17]:
df['Weather_Condition'].value_counts().idxmax()

'Clear'

In [18]:
# replace the missing 'City' values by the most frequent 
df["Weather_Condition"].replace(np.nan, "Clear", inplace=True)

In [19]:
df['Weather_Condition'].value_counts()

Clear                        884340
Fair                         547721
Mostly Cloudy                488094
Overcast                     382485
Partly Cloudy                344815
                              ...  
Drifting Snow                     1
Freezing Rain / Windy             1
Thunder and Hail                  1
Sand / Dust Whirls Nearby         1
Freezing Drizzle                  1
Name: Weather_Condition, Length: 127, dtype: int64

In [20]:
# replace Sunrise/Sunset column with the most occuring between "Night" and "Day"
df['Sunrise_Sunset'].value_counts().idxmax()

'Day'

In [21]:
# replace the missing 'NAN' values by the Day 
df["Sunrise_Sunset"].replace(np.nan, "Day", inplace=True)

In [22]:
missing_data3 = df.isnull()
for column in missing_data3.columns.values.tolist():
    print(column)
    print(missing_data3[column].value_counts())
    print("")

Source
False    3513617
Name: Source, dtype: int64

Severity
False    3513617
Name: Severity, dtype: int64

Start_Time
False    3513617
Name: Start_Time, dtype: int64

End_Time
False    3513617
Name: End_Time, dtype: int64

City
False    3513617
Name: City, dtype: int64

County
False    3513617
Name: County, dtype: int64

State
False    3513617
Name: State, dtype: int64

Visibility
False    3513617
Name: Visibility, dtype: int64

Weather_Condition
False    3513617
Name: Weather_Condition, dtype: int64

Sunrise_Sunset
False    3513617
Name: Sunrise_Sunset, dtype: int64



## At this point in the script, all the NAN values have wither been replaced by most occuring or average values of the column in question

In [27]:
df.head(5)
df.shape

(3513617, 10)

In [26]:
df.to_csv('Output.csv', index=True)